In [12]:
from selenium import webdriver
import time

In [13]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe')

In [14]:
# url in address window
browser.get('https://play.google.com/store/search?q=%ED%97%AC%EC%8A%A4%EC%BC%80%EC%96%B4%EC%95%B1&c=apps&hl=ko-KR')

In [15]:
browser.implicitly_wait(10)

In [16]:
import pandas as pd
import pymongo as mg

In [18]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['study_data_analytics']
collection = database['googlestore_heathcare']

# 서비스 앱에 대한 리스트
click_path = 'section > div > div > div > div'
company_list = browser.find_elements_by_css_selector(click_path)

len(company_list)          

for company in company_list:
    company.click()
    time.sleep(2)

    # 서비스 앱에 대한 정보 가져오기
    try:
        service_name = browser.find_element_by_css_selector('div > h1 > span').text
    except :
        service_name = str()
    try :    
        company_name = browser.find_element_by_css_selector('div.Il7kR > div > div > div > a > span').text
    except : 
        company_name = str()
    try :    
        app_information = browser.find_element_by_css_selector('div:nth-child(1) > c-wiz:nth-child(2) > div > section > div').text
    except :
        app_information = str()

    #### 리뷰 총 갯수 가져오기
    ## 리뷰 총 갯수 tag : div:nth-child(1) > div.g1rdde
    review_total_count_text = browser.find_element_by_css_selector('div:nth-child(1) > div.g1rdde').text
    
    # 댓글 팝업 띄우기 (해당 리뷰 없을 시 대비한 방어 코드 작성 필요)
    import re # reqexpress function
    result_list = re.findall(r'\d+', review_total_count_text)

    review_total_count = int(result_list[0]) # 리뷰 총 갯수
    
    # [service_name, company_name, app_information]

    # 댓글 팝업 띄우기(해당 리뷰 없을 시 대비한 방어 코드 작성 필요)
    review_flag = True
    try :
        browser.find_element_by_xpath('//*/div/div/button/*[contains(text(), "리뷰")]').click()    
    except :
        review_flag = False
    
    # 회사 서비스에 대한 전체 리뷰 가져오기
    serviceapp_columns_name = ['service_name', 'content', 'date', 'stars']
    reviews_list = list()
    if review_flag :
        #팝업 띄우기
        reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
        len(reviews_bundle)    

        #팝업 속 댓글 전체 표시
        review_standard_count_per = 40

        loop_count_int = int(review_total_count / review_standard_count_per)
        for count in range(1, loop_count_int+1) :
            try :
                reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
                print('current reviews_bundle count : {}'.format(len(reviews_bundle)))
                reviews_bundle[len(reviews_bundle)-1].click()
                # print('current reviews_bundle count : {}'.format(len(reviews_bundle)))
                time.sleep(3)
            except :
                print('pass')
                pass
        print('Done', len(reviews_bundle))

        # 팝업 댓글 수정
        for review_bundle in reviews_bundle:
            content = review_bundle.find_element_by_css_selector('div.h3YV2d').text
            date = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > span').text
            stars = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > div').get_attribute('aria-label')
            review_list = [service_name, content, date, stars]
            reviews_list.append(review_list)  # 리뷰에 대한 모든 것(특정 회사 서비스에 대한)

        browser.back() # 팝업 닫기
        
        # 저장 위한 작업
        df_reviews = pd.DataFrame(data=reviews_list, columns=serviceapp_columns_name)
        data_dict = df_reviews.to_dict(orient='records')
        collection.insert_many(data_dict)
        
    print('service_name : {}, review count : {}'.format(service_name, len(reviews_list)))
    browser.back() # 리스트로 돌아가기
    time.sleep(3)
print('Done')

current reviews_bundle count : 40
current reviews_bundle count : 80
current reviews_bundle count : 120
pass
Done 120
service_name : Samsung Health(삼성 헬스), review count : 120


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=116.0.5845.180)


In [ ]:
browser.quit()